# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
opened_df = pd.read_csv('output_data/weather_data.csv', index_col ='Unnamed: 0')
opened_df


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,datetime
0,Minas,-34.38,-55.24,11.92,76,97,4.10,12.22,UY,1593809103
1,Rikitea,-23.12,-134.97,21.67,71,41,8.49,21.67,PF,1593809104
2,Mayo,38.89,-76.51,33.24,55,1,3.60,34.44,US,1593809189
3,Severo-Kuril'sk,50.68,156.12,9.66,68,28,6.13,9.66,RU,1593809105
4,Rabo de Peixe,37.80,-25.58,21.00,73,40,3.10,21.00,PT,1593809190
...,...,...,...,...,...,...,...,...,...,...
539,Nacala,-14.54,40.67,20.76,79,15,4.56,20.76,MZ,1593809306
540,Ati,13.22,18.34,34.97,38,28,4.26,34.97,TD,1593809307
541,Kingsport,36.55,-82.56,31.94,38,40,1.33,32.22,US,1593809279
542,Sibu,2.30,111.82,23.93,94,40,1.50,24.00,MY,1593809307


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
opened_df.dropna()

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = opened_df[["lat", "long"]]

# set humidity values
humidity = opened_df["humidity"].astype(float)




In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# narrow by: temp between 70 and 80, windspeed less than 5, 0 cloudiness
# create new df, drop rest of rows
vacay_df = opened_df.loc[(opened_df["max temp"] > 25) & (opened_df["max temp"] < 30) & (opened_df["windspeed"] < 10) & (opened_df["cloudiness"] < 10)]
vacay_df

,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,datetime
88,Sur,22.57,59.53,29.40,75,1,5.88,29.40,OM,1593809207
108,Jamestown,42.10,-79.24,28.62,58,1,3.60,29.44,US,1593808920
155,Rosarito,32.33,-117.03,27.33,48,5,8.20,28.00,MX,1593809223
159,Zhangaözen,43.30,52.80,25.25,17,0,3.42,25.25,KZ,1593809136
161,Laguna,38.42,-121.42,27.61,36,1,3.60,28.33,US,1593809194
176,Avarua,-21.21,-159.78,26.00,78,5,2.60,26.00,CK,1593808938
178,Great Falls,47.50,-111.30,26.44,27,1,1.23,27.78,US,1593809228
197,Hilo,19.73,-155.09,26.86,54,1,2.60,29.00,US,1593809231
278,Umluj,25.02,37.27,29.43,42,0,1.83,29.43,SA,1593809249
294,Guerrero Negro,27.98,-114.06,28.32,40,0,5.87,28.32,MX,1593809253


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacay_df
hotel_df["Hotel Name"] = ""
hotel_df

/Users/marksutton/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,datetime,Hotel Name
88,Sur,22.57,59.53,29.40,75,1,5.88,29.40,OM,1593809207,
108,Jamestown,42.10,-79.24,28.62,58,1,3.60,29.44,US,1593808920,
155,Rosarito,32.33,-117.03,27.33,48,5,8.20,28.00,MX,1593809223,
159,Zhangaözen,43.30,52.80,25.25,17,0,3.42,25.25,KZ,1593809136,
161,Laguna,38.42,-121.42,27.61,36,1,3.60,28.33,US,1593809194,
176,Avarua,-21.21,-159.78,26.00,78,5,2.60,26.00,CK,1593808938,
178,Great Falls,47.50,-111.30,26.44,27,1,1.23,27.78,US,1593809228,
197,Hilo,19.73,-155.09,26.86,54,1,2.60,29.00,US,1593809231,
278,Umluj,25.02,37.27,29.43,42,0,1.83,29.43,SA,1593809249,
294,Guerrero Negro,27.98,-114.06,28.32,40,0,5.87,28.32,MX,1593809253,


In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

#Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["lat"]
    lng = row["long"]
    #print(lat)
    #print(lng)

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

/Users/marksutton/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,datetime,Hotel Name
88,Sur,22.57,59.53,29.40,75,1,5.88,29.40,OM,1593809207,Sur Plaza Hotel
108,Jamestown,42.10,-79.24,28.62,58,1,3.60,29.44,US,1593808920,Hampton Inn & Suites Jamestown
155,Rosarito,32.33,-117.03,27.33,48,5,8.20,28.00,MX,1593809223,Hotel Festival Plaza Rosarito
159,Zhangaözen,43.30,52.80,25.25,17,0,3.42,25.25,KZ,1593809136,K.korabay 28A
161,Laguna,38.42,-121.42,27.61,36,1,3.60,28.33,US,1593809194,Holiday Inn Express & Suites Elk Grove Central...
176,Avarua,-21.21,-159.78,26.00,78,5,2.60,26.00,CK,1593808938,Paradise Inn
178,Great Falls,47.50,-111.30,26.44,27,1,1.23,27.78,US,1593809228,La Quinta Inn & Suites by Wyndham Great Falls
197,Hilo,19.73,-155.09,26.86,54,1,2.60,29.00,US,1593809231,Hilo Hawaiian Hotel
278,Umluj,25.02,37.27,29.43,42,0,1.83,29.43,SA,1593809249,Moon light Furnished Units
294,Guerrero Negro,27.98,-114.06,28.32,40,0,5.87,28.32,MX,1593809253,"Casa Laguna, Bed & Breakfast"


In [8]:
hotel_df_10 = hotel_df.head(10)
hotel_df_10

,city,lat,long,temp,humidity,cloudiness,windspeed,max temp,country,datetime,Hotel Name
88,Sur,22.57,59.53,29.40,75,1,5.88,29.40,OM,1593809207,Sur Plaza Hotel
108,Jamestown,42.10,-79.24,28.62,58,1,3.60,29.44,US,1593808920,Hampton Inn & Suites Jamestown
155,Rosarito,32.33,-117.03,27.33,48,5,8.20,28.00,MX,1593809223,Hotel Festival Plaza Rosarito
159,Zhangaözen,43.30,52.80,25.25,17,0,3.42,25.25,KZ,1593809136,K.korabay 28A
161,Laguna,38.42,-121.42,27.61,36,1,3.60,28.33,US,1593809194,Holiday Inn Express & Suites Elk Grove Central...
176,Avarua,-21.21,-159.78,26.00,78,5,2.60,26.00,CK,1593808938,Paradise Inn
178,Great Falls,47.50,-111.30,26.44,27,1,1.23,27.78,US,1593809228,La Quinta Inn & Suites by Wyndham Great Falls
197,Hilo,19.73,-155.09,26.86,54,1,2.60,29.00,US,1593809231,Hilo Hawaiian Hotel
278,Umluj,25.02,37.27,29.43,42,0,1.83,29.43,SA,1593809249,Moon light Furnished Units
294,Guerrero Negro,27.98,-114.06,28.32,40,0,5.87,28.32,MX,1593809253,"Casa Laguna, Bed & Breakfast"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_10.iterrows()]
locations_hotel = hotel_df_10[["lat", "long"]]
#print(hotel_info)

In [10]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
marker_layer = gmaps.marker_layer(locations_hotel, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))